# First-order logic (v1)

## Variables

In [1]:
type Name = String
newtype Var = Var Name deriving (Eq,Ord) -- Ord instance necessary for list to set

instance Show Var where
  show (Var v) = v

## Formulas

In [2]:
data Formula = Atomic String [Var]
    | Neg Formula
    | Formula `Impl` Formula
    | Formula `Conj` Formula
    | Formula `Disj` Formula
    | Forall Var Formula
    | Exists Var Formula
    deriving Eq

In [3]:
at :: String -> String -> Formula
at p v = Atomic p [Var v]

forall :: String -> Formula -> Formula
forall v = Forall (Var v)

ex :: String -> Formula -> Formula
ex v = Exists (Var v)

### An example

"Every dog barks"

for all x(dog(x) -> barks(x))

In [4]:
_allDogsBark = forall "x" $ at "dog" "x" `Impl` at "bark" "x"

## Collecting free variables

In [5]:
{-# LANGUAGE OverloadedLists #-}

import qualified Data.Set as S

aSet :: S.Set Int
aSet = [1,2,1,3]

aSet

[1,2,3] `S.union` [1,2,4]

[1,2,3] `S.intersection` [1,2,4]

S.fromList [1,2] == S.fromList [2,1]

fromList [1,2,3]

fromList [1,2,3,4]

fromList [1,2]

True

In [6]:
import qualified Data.Set as S
import Data.Set (union)

freeVars :: Formula -> S.Set Var
freeVars (Atomic p vs) = S.fromList vs
freeVars (Neg p) = freeVars p
freeVars (p `Conj` q) = freeVars p `union` freeVars q
freeVars (p `Disj` q) = freeVars p `union` freeVars q
freeVars (p `Impl` q) = freeVars p `union` freeVars q
freeVars (Forall v p) = S.delete v (freeVars p)

## Recursive show

In [7]:
showVList :: [Var] -> String
showVList vs = "(" ++ showVs vs ++ ")" where
    showVs [] = ""
    showVs [v] = show v ++ showVs []
    showVs (v:vs) = show v ++ "," ++ showVs vs

In [8]:
showVList []

"()"

In [9]:
putStrLn $ showVList [Var "x", Var "y", Var "z"]

(x,y,z)

In [10]:
instance Show Formula where
  show (Atomic p vs) = p ++ showVList vs
  show (p `Impl` q) = "(" ++ show p ++ " -> " ++ show q ++ ")"
  show (p `Conj` q) = "(" ++ show p ++ " && " ++ show q ++ ")"
  show (p `Disj` q) = "(" ++ show p ++ " || " ++ show q ++ ")"
  show (Forall v p) = "A" ++ show v ++ show p
  show (Exists v p) = "E" ++ show v ++ show p

In [11]:
_allDogsBark

Ax(dog(x) -> bark(x))

In [12]:
freeVars _allDogsBark

fromList []

In [13]:
_freeTest = Forall (Var "x") $ Atomic "dog" [Var "x"] `Impl` Atomic "bark" [Var "z"] -- Ax(dog(x) -> bark(z))

In [14]:
freeVars _freeTest

fromList [z]

In [15]:
isClosed :: Formula -> Bool
isClosed p = S.null (freeVars p)

In [16]:
isClosed _allDogsBark

True

In [17]:
isClosed _freeTest

False

## Semantics

Let's use integers as our individuals, and define some predicates and relations as functions.

## Model

Watch out for undefinedness!

In [18]:
newtype Entity = E Int deriving (Eq,Show,Ord)

domE :: S.Set Entity
domE = S.fromList $ E <$> [1..10]

## Assignment

An assignment in predicate logic is a mapping from variables to individuals.

In [19]:
import qualified Data.Map as M

type Assignment = M.Map Var Entity

To give an interpretation, let's first define a function to gather *all* variables.

In [20]:
allVars :: Formula -> S.Set Var
allVars s = case s of
  (Atomic p vs) -> S.fromList vs
  (Neg p) -> allVars p
  (p `Impl` q) -> allVars p `S.union` allVars q
  (p `Conj` q) -> allVars p `S.union` allVars q
  (p `Disj` q) -> allVars p `S.union` allVars q
  (Forall v p) -> v `S.insert` allVars p
  (Exists v p) -> v `S.insert` allVars p

In [21]:
_allDogsBark
allVars _allDogsBark

Ax(dog(x) -> bark(x))

fromList [x]

Given a set of variables, and a domain of entities, we can generate all possible assignments as follows: 

In [22]:
import Control.Monad (replicateM)

mkAssignments :: [Var] -> [Entity] -> S.Set Assignment
mkAssignments vs d = S.fromList [M.fromList $ zip vs es | es <- replicateM (length vs) d]

In [23]:
mkAssignments (Var <$> ["x","y","z"]) (E <$> [1,2])

fromList [fromList [(x,E 1),(y,E 1),(z,E 1)],fromList [(x,E 1),(y,E 1),(z,E 2)],fromList [(x,E 1),(y,E 2),(z,E 1)],fromList [(x,E 1),(y,E 2),(z,E 2)],fromList [(x,E 2),(y,E 1),(z,E 1)],fromList [(x,E 2),(y,E 1),(z,E 2)],fromList [(x,E 2),(y,E 2),(z,E 1)],fromList [(x,E 2),(y,E 2),(z,E 2)]]

We need an interpretation function to map predicate symbols to boolean functions. We'll assume that anything not provided by the lexicon is tautological.

In [24]:
oddP :: [Entity] -> Bool
oddP [E n] = odd n
oddP _ = undefined

evenP :: [Entity] -> Bool
evenP [E n] = even n
evenP _ = undefined

isEqualR :: [Entity] -> Bool
isEqualR [E n, E n'] = n == n'
isEqualR _ = undefined

evenlyDivisibleR :: [Entity] -> Bool
evenlyDivisibleR [E n,E n'] = (n `rem` n') == 0
evenlyDivisibleR _ = undefined


In [25]:
type I = String -> [Entity] -> Bool

lexicon :: String -> [Entity] -> Bool
lexicon "odd" = oddP
lexicon "even" = evenP
lexicon "evenlyDivisible" = evenlyDivisibleR
lexicon "isEqual" = isEqualR
lexicon _ = const True

Now we can write a function to interpret formulas of predicate logic. We can re-use our lookup function.

In [38]:
import qualified Data.Map as M

eval :: I -> Dom -> Assignment -> Formula -> Bool
eval i d g (Atomic p vs) = i p [ g M.! v | v <- vs]
eval i d g (Neg p) = not $ eval i d g p
eval i d g (p `Conj` q) = eval i d g p && eval i d g q
eval i d g (p `Disj` q) = eval i d g p || eval i d g q
eval i d g (p `Impl` q) = not (eval i d g p) || eval i d g q
eval i d g (Exists v p) = undefined
eval i d g (Forall v p) = undefined

In [39]:
_xIsEven = Atomic "even" [Var "x"]
eval lexicon domE [(Var "x", E 2)] _xIsEven
eval lexicon domE [(Var "x", E 3)] _xIsEven

True

False

In order to define existential/universal quantification, we need a notion of *assignment modification*.

In [40]:
modify :: Assignment -> Var -> Entity -> Assignment
modify g v x = M.insert v x g

In [41]:
modify [(Var "x", E 1), (Var "y", E 2)] (Var "y") (E 1)

fromList [(x,E 1),(y,E 1)]

We'll also define generalized disjunction and conjunction recursively. Note that `True` is the identity element for conjunction, and `False` is the identity element for disjunction.

In [42]:
conjoin :: [Bool] -> Bool
conjoin [] = True
conjoin (t:ts) = t && conjoin ts

disjoin :: [Bool] -> Bool
disjoin [] = False
disjoin (t:ts) = t || disjoin ts

conjoin [True,False,True]
disjoin [True,False,True]

Line 2: Use foldr
Found:
conjoin [] = True
conjoin (t : ts) = t && conjoin ts
Why not:
conjoin ts = foldr (&&) True tsLine 6: Use foldr
Found:
disjoin [] = False
disjoin (t : ts) = t || disjoin ts
Why not:
disjoin ts = foldr (||) False ts

False

True

In [46]:
eval :: I -> Dom -> Assignment -> Formula -> Bool
eval i d g (Atomic p vs) = i p [ g M.! v | v <- vs]
eval i d g (Neg p) = not $ eval i d g p
eval i d g (p `Conj` q) = eval i d g p && eval i d g q
eval i d g (p `Disj` q) = eval i d g p || eval i d g q
eval i d g (p `Impl` q) = not (eval i d g p) || eval i d g q
eval i d g (Exists v p) = disjoin [ eval i d (modify g v x) p | x <- S.toList d]
eval i d g (Forall v p) = conjoin [eval i d (modify g v x) p | x <- S.toList d]

y is evenly divisible by some even number.

In [47]:
_everythingEven = Forall (Var "x") $ Atomic "even" [Var "x"]
_xIsEven = Atomic "even" [Var "x"]

In [53]:
trueSet :: I -> Dom -> Formula -> [Assignment]
trueSet i d p = [g | g <- S.toList (mkAssignments (S.toList $ allVars p) (S.toList d)), eval i d g p]

In [54]:
trueSet lexicon domE _everythingEven

[]

In [55]:
trueSet lexicon domE _xIsEven

[fromList [(x,E 2)],fromList [(x,E 4)],fromList [(x,E 6)],fromList [(x,E 8)],fromList [(x,E 10)]]

In [56]:
_xEvenNotDiv = Atomic "even" [Var "x"] `Conj` Neg (Atomic "evenlyDivisible" [Var "x", Var "y"])

In [57]:
trueSet lexicon domE _xEvenNotDiv

[fromList [(x,E 2),(y,E 3)],fromList [(x,E 2),(y,E 4)],fromList [(x,E 2),(y,E 5)],fromList [(x,E 2),(y,E 6)],fromList [(x,E 2),(y,E 7)],fromList [(x,E 2),(y,E 8)],fromList [(x,E 2),(y,E 9)],fromList [(x,E 2),(y,E 10)],fromList [(x,E 4),(y,E 3)],fromList [(x,E 4),(y,E 5)],fromList [(x,E 4),(y,E 6)],fromList [(x,E 4),(y,E 7)],fromList [(x,E 4),(y,E 8)],fromList [(x,E 4),(y,E 9)],fromList [(x,E 4),(y,E 10)],fromList [(x,E 6),(y,E 4)],fromList [(x,E 6),(y,E 5)],fromList [(x,E 6),(y,E 7)],fromList [(x,E 6),(y,E 8)],fromList [(x,E 6),(y,E 9)],fromList [(x,E 6),(y,E 10)],fromList [(x,E 8),(y,E 3)],fromList [(x,E 8),(y,E 5)],fromList [(x,E 8),(y,E 6)],fromList [(x,E 8),(y,E 7)],fromList [(x,E 8),(y,E 9)],fromList [(x,E 8),(y,E 10)],fromList [(x,E 10),(y,E 3)],fromList [(x,E 10),(y,E 4)],fromList [(x,E 10),(y,E 6)],fromList [(x,E 10),(y,E 7)],fromList [(x,E 10),(y,E 8)],fromList [(x,E 10),(y,E 9)]]

In [58]:
_somethingOdd = Exists (Var "x") $ Atomic "odd" [Var "x"]

In [59]:
trueSet lexicon domE _somethingOdd

[fromList [(x,E 1)],fromList [(x,E 2)],fromList [(x,E 3)],fromList [(x,E 4)],fromList [(x,E 5)],fromList [(x,E 6)],fromList [(x,E 7)],fromList [(x,E 8)],fromList [(x,E 9)],fromList [(x,E 10)]]

# Higher Order Abstract Syntax

In [ ]:
data Formula = Atomic String [Var]
    | Neg Formula
    | Formula `Impl` Formula
    | Formula `Conj` Formula
    | Formula `Disj` Formula
    | Forall (Var -> Formula)
    | Exists (Var -> Formula)

## TODO redo interpretation using this new schema